# UKAN 2023 


## Challenge #1 

###  Marine acoustic sensing using repurposed fibre optic cables

AIS data show that multiple vessels pass above the two fibre optic ables during the 4 day recording period.

The AIS data have been downloaded and stored as a CSV file.

We have also extracted likely time windows where the vessels are close to the cable

The challenge is

1) Find as many vessels as you can

2) Characterize the vessels e.g. speed, position etc


In [ ]:
import math
import glob
import scipy
import scipy.io
import matplotlib.dates

import pandas            as pd
import numpy             as np
import matplotlib.pyplot as plt

from  scipy.signal       import butter, lfilter
from  datetime           import datetime

def applyCosineTaper(Data,TaperLength=20):

    numberOfSamples = Data.shape[-1]
    taper           = np.sin(0.5*np.pi*np.arange(0,TaperLength)/(TaperLength-1))

    window     = np.ones(numberOfSamples)
    window[0:TaperLength] = taper
    window[-TaperLength:] = taper[::-1]

    return Data*window

def butter_bandpass(lowcut, highcut, fs, order=5):
    
    if lowcut <= 0:
        return butter(order, highcut, fs=fs, btype='low', analog=False)
    else:
        return butter(order, [lowcut, highcut], fs=fs, btype='band')

def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y


# Read AIS, North & South Cable Data

Here we take in the AIS data to be able to plot the paths that the vessels have taken to see if the go near the cables.
We then take this data to directly compare them and track which vessels have passed over the cables and exactly when that occured.

The AIS data is stored in a Pandas dataframe called df_Ais

The South cable geometry is stored in a Pandas dataframe called df_SouthCable.

The North cable geometry is stored in a Pandas dataframe called df_NorthCable.


In [ ]:
AisFilename = r'../../Documentation/AisData/AIS_167528271268456143_461-1675282713663.csv'
df_Ais = pd.read_csv(AisFilename) 
df_Ais['DateTimeUTC'] = pd.to_datetime(df_Ais['BaseDateTime'])

df_SouthCable= pd.read_csv(r'../../Documentation/CableGeometries/SouthCable_Geometry.csv')
df_SouthCable.describe()

df_NorthCable= pd.read_csv(r'../../Documentation/CableGeometries/NorthCable_Geometry.csv')
df_NorthCable.describe()


# Vessels travelling near the cables
Then we begin to extract the details about the journeys of the different vessels over the course of the 5 days so that we can plot that in relation to each other as well as the cables.

This allows for a visual to see what vessels travelled near the the location of the cables which we can later use to determine which vessels crossed over the cables.

In [ ]:
for day in range(1,6):                                              # Loop over the day 
     
    dd        = str(day).zfill(2)
    
    startTime = pd.to_datetime('2021-11-'+dd+'T00:00:01')
    endTime   = pd.to_datetime('2021-11-'+dd+'T23:59:59')

                                                                    # Filter the AIS data by time 
                                                                    # and geographic extent
    df_Ais2 = df_Ais[(df_Ais['DateTimeUTC'] > startTime) & (df_Ais['DateTimeUTC'] < endTime) &\
                     (df_Ais['LON']         > -125.1   ) & (df_Ais['LON']         < -123.5 ) &\
                     (df_Ais['LAT']         > 44.5     ) & (df_Ais['LAT']         < 45.5   ) ]

    plt.figure(figsize=(8,8))                                       # Create a plot of the vessel tracks                                                             

    plt.plot(df_Ais2['LON'],df_Ais2['LAT'],'k.',markersize=0.05,alpha=0.1)
    for immsi,mmsi in enumerate(np.unique(df_Ais2['MMSI'].values)):
        colors = ['b','g','r','c','m','y']
    
        c = colors[immsi%len(colors)]
                       
        mask           = df_Ais2 ['MMSI'       ] == mmsi
        vesselEasting  = df_Ais2 ['Easting'    ][mask].values
        vesselNorthing = df_Ais2 ['Northing'   ][mask].values
        vesselTime     = df_Ais2 ['DateTimeUTC'][mask].values
        
        indices        = np.argsort(vesselTime-np.min(vesselTime))
        vesselTime     = vesselTime     [indices]
        vesselEasting  = vesselEasting  [indices]
        vesselNorthing = vesselNorthing [indices]

        time0          = vesselTime[0]

        t = np.array([(t-time0)/ np.timedelta64(1, 's') for t in vesselTime])

        ti = np.arange(t[0],t[-1],15)
        xi = np.interp(ti,t,vesselEasting)
        yi = np.interp(ti,t,vesselNorthing)

        ti = np.array( [(time0+np.timedelta64(tii.astype('int'),'s')) for tii in ti])
        
        plt.plot(xi-df_SouthCable['Easting'][0],
                 yi-df_SouthCable['Northing'][0],'-'+c,alpha=0.5,linewidth=3)
        plt.plot(vesselEasting-df_SouthCable['Easting'][0],
                 vesselNorthing-df_SouthCable['Northing'][0],'s'+c,alpha=0.5,markersize=3)
        
        if vesselNorthing[0]-df_SouthCable['Northing'][0] > -60e3:
            plt.text(vesselEasting [0]-df_SouthCable['Easting' ][0],
                     vesselNorthing[0]-df_SouthCable['Northing'][0],str(mmsi),rotation=45)
        
        if vesselNorthing[-1]-df_SouthCable['Northing'][0] > -60e3:
            plt.text(vesselEasting [-1]-df_SouthCable['Easting' ][0],
                     vesselNorthing[-1]-df_SouthCable['Northing'][0],str(mmsi),rotation=45)
        
    plt.plot(df_SouthCable['Easting']-df_SouthCable['Easting'][0],
             df_SouthCable['Northing']-df_SouthCable['Northing'][0],'b-')
    plt.plot(df_NorthCable['Easting']-df_SouthCable['Easting'][0],
             df_NorthCable['Northing']-df_SouthCable['Northing'][0],'b-')
    plt.title('Day '+str(day)+'   ['+str(startTime)+']')
    plt.grid()
    plt.xlim(-120.e3,5e3)
    plt.ylim(-62.5e3,62.5e3)
    plt.savefig('./Ais_Day'+str(day)+'.png')
    plt.show()

# Find Vessel Crossings

In this section we take the AIS data and compare it directly against the cable location and geometry to find which vessels directly travelled over the cable.

This is so those timings could be used so that we know where to look in the cable data as to where events like vessel noise may occur.

The information is stored in a Pandas datframe called df_Closest.

In [ ]:
plt.figure(figsize=(8,4))

vesselsOfInterest = np.unique(df_Ais['MMSI'])

listOfClosest = []
for mmsi in vesselsOfInterest:

    mask           = df_Ais ['MMSI'       ] == mmsi
    vesselEasting  = df_Ais ['Easting'    ][mask].values
    vesselNorthing = df_Ais ['Northing'   ][mask].values
    vesselTime     = df_Ais ['DateTimeUTC'][mask].values

    vesselLength   = df_Ais ['Length'     ][mask].values[0]
    vesselType     = df_Ais ['Type'       ][mask].values[0]

    if np.isnan(vesselLength):
        
        vesselLength = 0
    
    if vesselEasting.shape[0] < 2: # Not enough data for vessel tracking

        continue

    indices        = np.argsort(vesselTime-np.min(vesselTime))
    vesselTime     = vesselTime     [indices]
    vesselEasting  = vesselEasting  [indices]
    vesselNorthing = vesselNorthing [indices]

    time0          = vesselTime[0]

    t = np.array([(t-time0)/ np.timedelta64(1, 's') for t in vesselTime])

    ti = np.arange(t[0],t[-1],15)
    xi = np.interp(ti,t,vesselEasting)
    yi = np.interp(ti,t,vesselNorthing)

    ti = np.array( [(time0+np.timedelta64(tii.astype('int'),'s')) for tii in ti])
    
    plt.figure(figsize=(12.5,5))
    for l in [(df_SouthCable,'South'),(df_NorthCable,'North')]:
    
        df_Cable  = l[0]
        cableName = l[1]

        cableEasting  = df_Cable['Easting' ].values
        cableNorthing = df_Cable['Northing'].values
        cableDepth    = df_Cable['Depth'   ].values

        times      = []
        distances  = []
        depths     = []
        location   = []
        for j in range(ti.shape[0]):

            ddd = np.hypot(cableEasting  - xi [j],\
                           cableNorthing - yi [j]) 

            location  .append( np.argmin(ddd) )
            distances .append( np.min   (ddd) )
            depths    .append( -1*cableDepth[location[-1]] )
            times     .append( (ti[j]-time0 ) // np.timedelta64(1, 's') )

        idd                       = np.argmin(distances)
        timeOfClosestApproach     = ti       [ idd ]
        eastingOfClosestApproach  = xi       [ idd ]
        northingOfClosestApproach = yi       [ idd ]
        locOfClosestApproach      = location [ idd ]
        distOfClosestApproach     = np.min   (distances)

        if min(distances) < 10.0e3:

            print(mmsi,
                  timeOfClosestApproach,
                  eastingOfClosestApproach,
                  northingOfClosestApproach)

            listOfClosest.append( [mmsi,
                                   vesselType,
                                   vesselLength,
                                   timeOfClosestApproach,
                                   distOfClosestApproach,
                                   locOfClosestApproach,
                                   eastingOfClosestApproach,
                                   northingOfClosestApproach,
                                   cableName,
                                   pd.to_datetime(timeOfClosestApproach).strftime('%Y%m%d_%H%M%S')] )

            if False:
                
                plt.figure(figsize=(8,3))

                plt.subplot(2,1,1)
                plt.plot(ti,distances,'k.-')
                plt.ylim(0,10e3)
                plt.gcf().autofmt_xdate()
                plt.grid()

                plt.subplot(2,1,2)
                plt.plot(ti,depths,'k.-')
                plt.ylim(0,1e3)
                #plt.xlim(0,(endTime-startTime).total_seconds())
                #plt.xticks(np.arange(0,(endTime-startTime).total_seconds(),60*60*24))
                #plt.yticks(np.arange(0,10e3,2e3))
                plt.gcf().autofmt_xdate()
                plt.grid()

                plt.suptitle(str(mmsi)+' : '+str(timeOfClosestApproach))
                plt.savefig('./time_'+str(mmsi)+'png')
                plt.show()

            
            mask = df_Ais ['MMSI'] == mmsi
            
            plt.plot(xi -cableEasting [0],
                     yi-cableNorthing[0],'r.',
                     alpha=0.1,
                     markersize=1)
            plt.plot(vesselEasting  - cableEasting [0],
                     vesselNorthing - cableNorthing[0],'rs',
                     alpha=0.5,
                     markersize=2)
            plt.plot(df_SouthCable['Easting' ] - df_SouthCable['Easting' ][0],
                     df_SouthCable['Northing'] - df_SouthCable['Northing'][0],
                     'b-',alpha=0.25)
            plt.plot(df_NorthCable['Easting' ] - df_NorthCable['Easting' ][0],
                     df_NorthCable['Northing'] - df_NorthCable['Northing'][0],
                     'b-',alpha=0.25)
            plt.plot(cableEasting  - cableEasting  [0],
                     cableNorthing - cableNorthing [0],
                     'b-',alpha=0.5)
            
            plt.plot(cableEasting [locOfClosestApproach]  - cableEasting  [0],
                     cableNorthing[locOfClosestApproach] - cableNorthing [0],
                     'bs',alpha=1.0)
            
            dtt = pd.to_datetime(str(timeOfClosestApproach)).strftime('%d.%m.%Y %H:%M:%S')
            
            plt.text(cableEasting [locOfClosestApproach]  - cableEasting  [0],
                     cableNorthing[locOfClosestApproach] - cableNorthing [0]+300,
                     dtt[0:2]+"\n"+dtt[10:])
                                     
    plt.xlim(-100e3,0)
    plt.ylim(-20e3,20e3)
    plt.xticks(np.arange(-100e3,  0,10e3))
    plt.yticks(np.arange(-20e3,20e3,10e3))
    plt.grid()
    plt.title('MMSI'+str(mmsi)+' '+str(int(vesselLength))+'m '+vesselType+' '+dtt[0:10])
    plt.xlabel('Easting [km]')
    plt.ylabel('Northing [km]')
    plt.savefig('./track_'+str(mmsi)+'png')
    plt.show()

df_Closest= pd.DataFrame(listOfClosest,columns=['MMSI',
                                                'Type',
                                                'Length',
                                                'timeOfClosestApproach',
                                                'distOfClosestApproach',
                                                'locationOfClosestApproach',
                                                'eastingOfClosestApproach',
                                                'northingOfClosestApproach',
                                                'Cable',
                                                'datetime'])

if False:
    
    df_Closest.to_csv(r'../../Documentation/AisData/ClosestVessels.csv')

# Vessel crossing example

We can use the dates and times which have vessel crossings to then look for the actual vessel event in the recorded data.

Below, we have an example of vessel 255613000 which crosses the South cable at 1:23 on 3/11/2021.

This is the example shown in Wilcox et al., 2020.


In [ ]:
dataDir = r'../../Data/Vessels/MatlabFormat/'

files   = sorted(glob.glob(dataDir+'Slx_South_20211103012424_30_2_200.mat'))

print('Number of mat Files: ',len(files))

for file in files:
    
    data = scipy.io.loadmat(file)['data']
    numberOfChannels,\
    numberOfSamples = data.shape

    samplingFrequency = float(file[0:-4].split("_")[-1])
    gaugeLength       = float(file[0:-4].split("_")[-2])
    channelSpacing    = float(file[0:-4].split("_")[-3])
    
    timeStamp         = file[file.rfind("_2021")+1:file.rfind("_2021")+15]
    dateTime          = datetime.strptime(timeStamp, '%Y%m%d%H%M%S')
    
    mean1             = np.mean(data,axis=1)

    bdata             = butter_bandpass_filter(applyCosineTaper((data.T - mean1).T,400),\
                                               10, 60, fs=samplingFrequency, order=5)
    

In [ ]:
%matplotlib 

vMax = np.percentile(bdata[::,::],90)
plt.imshow(bdata[::,::],aspect='auto',cmap='bwr',vmin=-vMax,vmax=vMax)
plt.title(file)
plt.xlabel('Time [Samples]')
plt.ylabel('Channel')
plt.ylim(24000,27000)
plt.xlim(1000,1500)
plt.show()

# Your code after this cell !


- How many vessels can you find ?


- What is the smallest vessel you can find ?


- How far away can you detect a vessel from the cable ?


- What additional information can you find out about the vessel ?


**Have fun and good luck !**

*Other useful links*

[awesome-das: A curated list of DAS tools and resources on Github](https://github.com/DAS-RCN/awesome-das)

[DAS4Whale - A Python package to analyze DAS data for marine bioacoustics](https://github.com/leabouffaut/DAS4Whales)

[DASDAE - A python library for distributed fiber optic sensing](https://github.com/DASDAE/)


Hint 1: Bigger vessels are louder !

Hint 2: Look at the AIS data to check which vessels *actually* pass closest to the cable (the solid red dots on the plots above, the solid line is just extrapolated between the AIS positions)

Hint 3: Look at the paper's for ideas about how to improve the signal processing and what inforamtion you could extract.
